# Contents

In this notebook, I induce Rabi oscillation on the (12) subspace of the superconducting qutrit.

In [1]:
import os 
os.chdir('/Users/ngdnhtien/Codespace/PulsatingPulseShop/')

# Necessary dependencies
import qiskit
import qiskit_ibm_provider
import utility as ut
import numpy as np
import pickle 
import matplotlib.pyplot as plt
import json
import datetime

In [2]:
plt.rcParams['axes.linewidth'] = 1.0
%config InlineBackend.figure_formats = ['svg']
today = datetime.datetime.now()

In [3]:
provider = qiskit_ibm_provider.IBMProvider()
backend = provider.get_backend('ibm_brisbane')

qubit = 109
cbit = 0

num_qubits = int(qubit+1)
num_cbits = 1

weight = 1

rough_01_freq = backend.defaults().qubit_freq_est[qubit]
qubit_anharmonicty = backend.properties().qubits[qubit][3].value * 1e9
rough_12_freq = rough_01_freq + qubit_anharmonicty

print(r"f01 = "+f'{round(rough_01_freq/1e9, 4)}'+' GHz')
print(r'f12 = '+f'{round(rough_12_freq/1e9, 4)}'+' GHz')
print(r'anharmonicity = '+f'{round(qubit_anharmonicty/1e9, 4)}'+' GHz')

f01 = 4.985 GHz
f12 = 4.6779 GHz
anharmonicity = -0.3071 GHz


# Measurement pulse

In [32]:
shift_frequency = -0.40000000000000013
measure_amplitude = 0.1

with qiskit.pulse.build(backend=backend, name='Spectroscopy') as measure_schedule:
    #define channel
    meas_chan = qiskit.pulse.measure_channel(qubit)
    qiskit.pulse.shift_frequency(shift_frequency*1e6, meas_chan)
    ac_chan = qiskit.pulse.acquire_channel(qubit)
    mem_chan = qiskit.pulse.MemorySlot(0)
    #define pulse shape
    qiskit.pulse.acquire(1440, ac_chan, mem_chan)
    qiskit.pulse.play(qiskit.pulse.GaussianSquare(amp=measure_amplitude, duration=1440,
                                    sigma=32, width=1312,
                                    angle=1.330056315168335), meas_chan)
    #add delay
    qiskit.pulse.delay(1160, meas_chan)

# Residual population of $|2\rangle$

In [35]:
with qiskit.pulse.build(backend=backend) as x12_sched:
    drive_chan = qiskit.pulse.drive_channel(qubit)
    qiskit.pulse.set_frequency(rough_12_freq, drive_chan)
    qiskit.pulse.play(qiskit.pulse.Gaussian(duration=48, amp=0.402915014835218, sigma=int(48/4)), drive_chan)

x12_gate = qiskit.circuit.Gate('X12', weight, [])

circ_0 = qiskit.circuit.QuantumCircuit(num_qubits, num_cbits)
circ_0.x(qubit)
circ_0.append(x12_gate, [qubit])
circ_0.x(qubit)
circ_0.measure(qubit, cbit)
circ_0.add_calibration(x12_gate, [qubit], x12_sched)
circ_0.add_calibration('measure', [qubit], measure_schedule)

circ_1 = qiskit.circuit.QuantumCircuit(num_qubits, num_cbits)
circ_1.measure(qubit, cbit)
circ_1.add_calibration('measure', [qubit], measure_schedule)

circs = [circ_0, circ_1]

job = backend.run(circs, meas_level=1, meas_return='single', shots=4096)

In [34]:
job.status(), job.job_id() # 160

(<JobStatus.QUEUED: 'job is queued'>, 'ctzh2kjg22kg0084tst0')

In [36]:
job.status(), job.job_id() # 48

(<JobStatus.QUEUED: 'job is queued'>, 'ctzh2ykzzp40008h373g')

In [29]:
experiment = ut.DataAnalysis(experiment=job, average=False, shots=4096, qubit=0)
experiment.retrieve_data(average=False)

with open(f"./characterization/rabi12/qubit109/data/batch5_vary_rep_delay/residual_2.pkl", "wb") as f:
    pickle.dump(experiment.IQ_data, f)

# * Rabi oscillation with an additional $\pi^{01}$ pulse

### Experiment parameters

In [48]:
n = input('The duration must be a integer multiple of 8 and larger than 2: ')
dur = int(8*int(n))

range_amp = float(input('Range of amplitude sweep: '))
num_point = int(input('Number of data points: '))
num_shots = int(2**12)

print(f'Current multiple {n}')
print(f'Duration is {dur} dt (must be integer)')
print(f'Sweeping from -0.1 to {range_amp}...')

Current multiple 6
Duration is 48 dt (must be integer)
Sweeping from -0.1 to 1.0...


### Run script

In [42]:
with qiskit.pulse.build(backend=backend) as reset_sched:
    drive_chan = qiskit.pulse.drive_channel(qubit)
    qiskit.pulse.set_frequency(rough_12_freq, drive_chan)
    qiskit.pulse.play(qiskit.pulse.Gaussian(duration=48, amp=0.402915014835218, sigma=int(48/4)), drive_chan)

In [52]:
amplitudes = np.linspace(-0.1, range_amp, num_point)
rabi_gate = qiskit.circuit.Gate('rabi_gate', weight, [])
reset_2 = qiskit.circuit.Gate('reset_2', weight, [])

rabi_osc_circs = []

for amp in amplitudes:
    with qiskit.pulse.build(backend=backend) as rabi_sched:
        drive_chan = qiskit.pulse.drive_channel(qubit)
        qiskit.pulse.set_frequency(rough_12_freq, drive_chan)
        qiskit.pulse.play(qiskit.pulse.Gaussian(duration=dur, amp=amp, sigma=int(dur/4)), drive_chan)
    rabi_circ = qiskit.circuit.QuantumCircuit(num_qubits, num_cbits)
    rabi_circ.x(qubit) 
    rabi_circ.append(rabi_gate, [qubit])
    rabi_circ.x(qubit)
    rabi_circ.measure(qubit, cbit)
    # rabi_circ.append(reset_2, [qubit])
    # rabi_circ.add_calibration(reset_2, [qubit], reset_sched)
    rabi_circ.add_calibration(rabi_gate, [qubit], rabi_sched)
    rabi_osc_circs.append(rabi_circ)

rabi_job_single = backend.run(rabi_osc_circs, meas_level=1, meas_return='single', shots=num_shots)
print(rabi_job_single.job_id())

ctyz86g5yns0008s7feg


### Check status

In [55]:
rabi_job_single = provider.backend.retrieve_job('ctyz86g5yns0008s7feg') # rep_delay = 0
#rabi_job_single = provider.backend.retrieve_job('ctyyfw7m8gh0008egv20') # rep_delay = 0.0001
#rabi_job_single = provider.backend.retrieve_job('ctyyfyqfhad0008tjw00') # rep_delay = 0.0002

print(rabi_job_single.status())

JobStatus.DONE


###

 Save raw data

In [56]:
rabi_job_single = provider.backend.retrieve_job('ctyz86g5yns0008s7feg')
rabi_exp_single = ut.DataAnalysis(experiment=rabi_job_single, average=False, shots=num_shots, qubit=0)
rabi_exp_single.retrieve_data(average=False)

with open(f"./characterization/rabi12/qubit109/data/batch5_vary_rep_delay/{dur}_-0.1_{range_amp}_no_AR.pkl", "wb") as f:
    pickle.dump(rabi_exp_single.IQ_data, f)

# Check status

In [31]:
ran_job = None
for ran_job in provider.backend.jobs():
    print(str(ran_job.job_id()) + " " + str(ran_job.status()))

ctyyrzvh4j1g0082s5k0 JobStatus.CANCELLED
ctyyfyqfhad0008tjw00 JobStatus.DONE
ctyyfw7m8gh0008egv20 JobStatus.DONE
ctyyfr7fhad0008tjvzg JobStatus.DONE
ctyy658fhad0008tjvkg JobStatus.DONE
ctyy61rsvvng008sd9d0 JobStatus.DONE
ctyy4mj5yns0008s7ckg JobStatus.DONE
ctsh134ezn20008s8a7g JobStatus.DONE
ctrzd6c4gjh0008q0zkg JobStatus.DONE
ctryzqy4gzyg008zz3j0 JobStatus.CANCELLED


In [30]:
job = provider.backend.retrieve_job('ctyyrzvh4j1g0082s5k0')

job.cancel()

True